In [18]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 21000)

Dataset: mix-instruct_train_21000 found in cache, loading from cache ✅


In [19]:
from utility_functions.delift_se import get_delift_se_utility

utility, utility_name = get_delift_se_utility(prompts, references, ds_name)

Utility: mix-instruct_train_21000_delift-se not found in cache, computing now 🏃


Computing pairwise similarities: 100%|██████████| 3/3 [00:01<00:00,  2.82it/s]


Utility: mix-instruct_train_21000_delift-se computed and saved to cache ✅


In [ ]:
from subset import create_subset, get_subset

subset, subset_name = create_subset(utility, utility_name)

Subset: mix-instruct_train_21000_delift-se_0.3 not found in cache, computing now 🏃


In [12]:
from inference import generate_responses

# Phi3: 'microsoft/Phi-3-mini-128k-instruct'
# LLama: 'meta-llama/Llama-3.2-3B'
responses, generation_name = generate_responses(prompts, 'microsoft/Phi-3-mini-128k-instruct', ds_name, 'cuda:3', batch_size=64)

Generation: mix-instruct_train_21000_Phi-3-mini-128k-instruct found in cache, loading from cache ✅


In [15]:
from evaluation import compute_metrics
# For Phi3 (initial)
# Reported BGE: 0.80, ROUGE: 0.35
compute_metrics(responses, references, generation_name, device='cuda:0')

Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_rouge found in cache, loading from cache ✅


{'bge': 0.808487, 'rouge': 0.21496158928534306}

In [16]:
# For LLama 3.2 (initial)
# Reported BGE: 0.73, ROUGE: 0.28
responses_llama, generation_name_llama = generate_responses(prompts, 'meta-llama/Llama-3.2-3B', ds_name, 'cuda:3', batch_size=64)
compute_metrics(responses_llama, references, generation_name_llama, device='cuda:0')

Generation: mix-instruct_train_21000_Llama-3.2-3B found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Llama-3.2-3B_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Llama-3.2-3B_rouge found in cache, loading from cache ✅


{'bge': 0.7400304, 'rouge': 0.10360998123357584}

In [17]:
import numpy as np
import evaluate

rouge_metric = evaluate.load('rouge')
def calculate_evaluate_metric(predictions, references, score="rouge", return_invidiual=True):
    """
    Calculates the similarity (rouge, bleu, or bertscore) between the predictions and references

    Args:
        predictions: list of strings for the hypothesis
        references: list of strings for the reference
        score: one of "rouge", "bleu", "bertscore", "bge", "promedeus"
        return_invidiual: if True, it will return the individual scores for corresponding prediction-reference pairs
    Returns:
        np array of metrics of size 1x1 if return_individual is True, else 1x|predictions|
    """
    if not return_invidiual:
        predictions = [predictions]
        references = [references]
    else:
        predictions = [[p] for p in predictions]
        references = [[r] for r in references]


    if score == "rouge":
        sim_metric = rouge_metric
        metric_key = "rouge1"

    metrics = []
    for p, r in zip(predictions, references):
        if score == "bertscore":
            metrics.append(np.array(sim_metric.compute(predictions=p, references=r, lang="en")[metric_key]).mean())
        else:
            metrics.append(sim_metric.compute(predictions=p, references=r)[metric_key])
    return np.array(metrics)

calculate_evaluate_metric(responses_llama[:1000], references[:1000], return_invidiual=False)

array([0.10462405])